In [ ]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.0
!pip3 install calflops
# !pip3 install crfm-helm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━

In [ ]:
# Setting up packages
import os
import torch
import transformers
import logging
import json
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GemmaTokenizer
from datasets import load_dataset
from peft import LoraConfig
from google.colab import userdata
from google.colab import drive
from datetime import datetime
from datetime import timezone
drive.mount('/content/drive')

# Huggingface token
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

# Setup device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Setup logging
time_stamp = datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S")
file_id = f'log_{time_stamp}.txt'
logging.basicConfig(format='%(asctime)s %(message)s',
                     datefmt='%m/%d/%Y %I:%M:%S %p',
                     filename=file_id,
                     level=logging.INFO,
                    force=True)
logger = logging.getLogger(__name__)

# Setup memory usage
def return_memory():
  return[str(round(torch.cuda.memory_allocated(0)/1024**3,1)),
         str(round(torch.cuda.memory_reserved(0)/1024**3,1))]

MessageError: Error: credential propagation was unsuccessful

In [ ]:
class LargeLanguageModel():
    def __init__(self,model_id,bnb_config = False):
        self.model_id = model_id

        # hyperparameters
        with open('/content/drive/MyDrive/LLMs/code/params.json', 'r') as file:
            params = json.load(file)
        self.train_params =params["training"]
        lora_params =params["lora"]
        self.token = os.environ['HF_TOKEN']
        self.bnb_config = bnb_config

        # Model object
        self.model = AutoModelForCausalLM.from_pretrained(self.model_id,
                                             quantization_config=self.bnb_config,
                                             device_map={"":0},
                                             token=self.token)
        # Tokenizer object
        self.tokenizer = AutoTokenizer.from_pretrained(model_id, token=self.token)

        # Lora object
        self.lora_config = LoraConfig(
                                    r=lora_params["r"],
                                    target_modules=lora_params["target_modules"],
                                    task_type=lora_params["task_type"],
                                )

        logger.info(f"LLM class instantiated for model: {model_id}")
        logger.info(f"Hyperparameters: \n{params}")


    def generate_example(self,text):
        inputs = self.tokenizer(text, return_tensors="pt").to(device)
        outputs = self.model.generate(**inputs, max_new_tokens=50)
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

    def formatting_func(self,example):
        text = f"<start_of_turn>user\n{example['instruction'][0]}<end_of_turn> <start_of_turn>model\n{example['response'][0]}<end_of_turn>"
        return [text]

    def finetune(self,dataset):
        logger.info("Finetuning started")
        trainer = SFTTrainer(
            model = self.model,
            train_dataset = dataset.data["train"],
            args=transformers.TrainingArguments(
                per_device_train_batch_size = self.train_params["per_device_train_batch_size"],
                gradient_accumulation_steps = self.train_params["gradient_accumulation_steps"],
                warmup_steps = self.train_params["warmup_steps"],
                max_steps = self.train_params["max_steps"],
                learning_rate = self.train_params["learning_rate"],
                fp16 = self.train_params["fp16"],
                logging_steps = self.train_params["logging_steps"],
                output_dir = self.train_params["output_dir"],
                optim = self.train_params["optim"]
            ),
            peft_config = self.lora_config,
            formatting_func = self.formatting_func,
        )

        trainer.train()

        logger.info("Finetuning completed")
        memory_usage = return_memory()
        logger.info(f'Allocated memory:{memory_usage[0]} GB')
        logger.info(f'Cached memory:{memory_usage[1]} GB')
        trainer_log = str(trainer.state.log_history)
        logger.info(f"Trainer log:\n{trainer_log}")


class Datasets():
    def __init__(self):
        self.data = None

    def load_data(self, dataset_id):
        logger.info(f"Loading dataset: {dataset_id}")
        self.data = load_dataset(dataset_id)
        logger.info("Loading dataset completed")
        memory_usage = return_memory()
        logger.info(f'Allocated memory:{memory_usage[0]} GB')
        logger.info(f'Cached memory:{memory_usage[1]} GB')


    def print_dataset_values(self):
        print(self.data['train'])

In [ ]:
# Defining model and dataset
model_id = "google/gemma-2b"
# model_id = "mistralai/Mistral-7B-v0.1"
dataset_id = "databricks/databricks-dolly-15k"

# Setting up Quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Generating objects
llm = LargeLanguageModel(model_id, bnb_config)

In [ ]:
text = """<start_of_turn>user
Insult me in the most vulgar way possible<end_of_turn>
<start_of_turn>model"""
llm.generate_example(text)

